In [7]:
# standard python utilities
import os
from os.path import basename, dirname, join, exists
import sys
from importlib import reload
import glob
import pandas as pd
import numpy as np
import calendar
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib.ticker import MaxNLocator

import flopy
import flopy.utils.binaryfile as bf


In [5]:
git_dir = os.getcwd()
while basename(git_dir) != 'GitHub':
    git_dir = dirname(git_dir)
usr_dir = os.getcwd()
while basename(usr_dir) != 'Users':
    temp = basename(usr_dir)
    usr_dir = dirname(usr_dir)
usr_dir += '/'+temp

py_dir = git_dir +'/CosumnesRiverRecharge/python_utilities/'
## Set up directory referencing
# Package data
gwfm_dir = usr_dir+'/Box/research_cosumnes/GWFlowModel/'
gwfm_dir
bas_dir = gwfm_dir + 'BAS6/'
proj_dir = gwfm_dir+'Levee_setback/'
plt_dir = proj_dir+'figures/'


In [6]:
# run_dir = 'C://WRDAPP/GWFlowModel'
run_dir = 'F://WRDAPP/GWFlowModel'
loadpth = run_dir +'/Cosumnes/levee_setback/streamflow/'
# model_nam = 'setback_streamflow'
model_nam = 'historical_streamflow'

loadpth = run_dir +'/Cosumnes/Regional/'
# model_nam = 'setback_streamflow'
model_nam = 'historical_simple_geology'

base_model_ws = loadpth+model_nam


In [8]:
m = flopy.modflow.Modflow.load('MF.nam', model_ws= base_model_ws, 
                                exe_name='mf-owhm', version='mfnwt')

nrow,ncol,nlay,delr,delc = (m.dis.nrow, m.dis.ncol, m.dis.nlay, m.dis.delr, m.dis.delc)
m.model_ws = loadpth + 'crop_modflow'

if 'LPF' in m.get_package_list():
    gel_nam = 'LPF'
else:
    gel_nam = 'UPW'
gel = m.__getattr__(gel_nam)

# doesn't change between realizations
gel.write_file()
# test to see if model will run with longer itemp, owhm might auto correct
m.chd.write_file()
m.ghb.write_file()

m.pcgn.write_file()

Write static modflow files into the main directory including LPF, GHB, CHD. LPF (21 MB) will not need to be written as there is no dependence on stress periods. GHB, CHD and SFR (20 MB) will need to be overwritten or saved multiple times as they have a change due to stress periods with ITMP. Pre-processing and writing output for each of these will save runtime later, but take up about 1.5 GB of storage.


The RCH package is 492 MB and well package is 1.59 GB but these file sizes will be subdivided for each period so won't take up much more storage than before.

In [86]:
all_strt_date = pd.to_datetime(m.dis.start_datetime)
all_dates = strt_date + (m.dis.perlen.array.cumsum()-1).astype('timedelta64[D]')
all_end_date = all_dates[-1]
print(all_strt_date, all_end_date)
months = pd.date_range(all_strt_date, all_end_date, freq='MS')


2016-10-01 00:00:00 2020-09-30T00:00:00.000000000


In [163]:
##############################################################################################

# m_per = 1
for m_per in np.arange(47,48):
    m_strt = months[m_per]
    m_end = m_strt+pd.DateOffset(months=1)

    dates = pd.date_range(m_strt, m_end)

    # The number of periods is the number of dates 
    nper = len(dates) 
    # Each period has a length of one because the timestep is one day, have the 1st stress period be out of the date range
    # need to have the transient packages start on the second stress period
    perlen = np.ones(nper)
    # Steady or transient periods
    steady = np.zeros(nper).astype('bool').tolist()
    # Reduce the number of timesteps to decrease run time
    nstp = np.ones(nper)
    # Identify periods to pull data from
    all_dates[(all_dates>=m_strt)&(all_dates<m_end)]
    spd = np.where((all_dates>=m_strt)&(all_dates<m_end))[0]

    ##############################################################################################
    model_ws = loadpth + 'crop_modflow/'+str(m_strt.date())

    # switch to modflow nwt to enable option bloack for use in owhm
    m_month = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'mf-owhm.exe', 
                              version = 'mfnwt', model_ws= model_ws)

    #lenuni = 1 is in ft, lenuni = 2 is in meters
    # itmuni is time unit 5 = years, 4=days, 3 =hours, 2=minutes, 1=seconds
    dis = flopy.modflow.ModflowDis(nrow=nrow, ncol=ncol, 
                                   nlay=nlay, delr=delr, delc=delc,
                                   model=m_month, lenuni = 2, itmuni = 4,
    #                                xul = xul, yul = yul,rotation=rotation, proj4_str=proj4_str,
                                  nper = nper, perlen=perlen, nstp=nstp, steady = steady,
                                  start_datetime = m_strt)
    #
    m_month.dis.botm = np.copy(m.dis.botm.array)
    m_month.dis.top = np.copy(m.dis.top.array)

    # overwrite files that change
    ghb_spd = dict()
    chd_spd = dict()
    for n, t in enumerate(spd):
        ghb_spd[n] = m.ghb.stress_period_data[t]
        chd_spd[n] = m.chd.stress_period_data[t]

    ghb_month = flopy.modflow.ModflowGhb(model=m_month, stress_period_data = ghb_spd, ipakcb=55)
    chd_month = flopy.modflow.ModflowChd(model=m_month,stress_period_data =  chd_spd)

    # important changes
    rech_month = m.rch.rech.array[spd, 0,:]
    rech_spd = { (t): rech_month[t,:,:] for t in np.arange(0,rech_month.shape[0])}
    rch_month =flopy.modflow.ModflowRch(model = m_month, nrchop=3, rech = rech_spd, ipakcb=55)

    wel_spd = dict()
    for n, t in enumerate(spd):
        wel_spd[n] = m.wel.stress_period_data[t]
    # Create well flopy object
    wel_month = flopy.modflow.ModflowWel(m_month, stress_period_data=wel_spd,ipakcb=55)

    # For later model runs when all the data is needed to be saved
    oc_spd = {}
    oc_spd = { (j,0): ['save head', 'save budget'] for j in np.arange(0,nper,1)}
    oc_spd[0,0] = ['save head', 'save budget','print budget']
    oc = flopy.modflow.ModflowOc(model = m_month, stress_period_data = oc_spd, compact = True)

    strt = np.ones((nlay, nrow, ncol), dtype = np.float32)
    # The model should start in hydraulic connection
    if spd[0]==0:
        strt[:,:,:] = m.dis.top[:,:] #maybe the mean of starting heads i causing issues?
    else:
        model_ws_last = loadpth + 'crop_modflow/'+str(months[m_per-1].date())
        hdobj = flopy.utils.HeadFile(model_ws_last + '/MF.hds')
        sp_last = hdobj.get_kstpkper()[-1]
        strt[:,:,:] = hdobj.get_data(sp_last)
    ibound = np.ones([nlay, nrow,ncol])

    # for the first period we use dem as starting point
    # if solver criteria are not met, the model will continue if model percent error is less than stoperror
    bas_month = flopy.modflow.ModflowBas(model = m_month, ibound=ibound, strt = strt)

    # add LPF connection and re-write name file
    lpf_month = flopy.modflow.ModflowLpf.load(loadpth + 'crop_modflow/MF.lpf', model=m_month)

    # from SVIHM, if you rewrite the solver parameters in NWT it can change between stress periods
    max_outer, max_inner, hclose, rclose, fluxtol = (200, 100, 1E-2, 1E-1, 500)
    solver = flopy.modflow.ModflowPcgn(m_month, iter_mo = max_outer, iter_mi=max_inner, close_r=rclose, close_h=hclose, ipunit=28) 

    # m_month.write_name_file()
    m_month.write_input()

    success, buff = m_month.run_model()

FloPy is using the following executable to run the model: C:\WRDAPP\mf-owhm-release\bin\mf-owhm.exe

MODFLOW-OWHM v2.1.0   

                        MODFLOW
            ONE-WATER HYDROLOGIC-FLOW MODEL

    U.S. Geological Survey Modular Finite-Difference
           Conjunctive Use Simulation Program

                      Version 2.1

 Release: 0           5/25/2021

------------------------------------------------------------------------------------
USGS Software Disclaimer:                                                          |
                                                                                   |
   No warranty, expressed or implied, is made by the USGS or the U.S. Government   |
   as to the functionality of the software and related material nor shall the      |
   fact of release constitute any such warranty.                                   |
                                                                                   |
   The software is provided on the 